# Introduction : 
This notebook will study the influence of the reormalisation on the frequency renormalisation of a small junction 

## Libraries : 

In [1]:
from __future__ import division
import numpy as np
from numpy.linalg import inv
import scipy.linalg as lin
import scipy.constants as cst
import matplotlib.pyplot as plt
%matplotlib qt

## Intializing parameters : 

In [2]:
def initialize_parameters():
    # Physical constants and parameters
    phi0 = cst.hbar / (2 * cst.e)  # Flux quantum
    h = cst.h
    e = cst.e

    # Device-specific parameters (Shelender's Sample 8)
    N = 1498
    ELa = 66e6 * h
    Eja = N * ELa
    Cja = 45e-15 * 1.890 * 0.207
    ECa = e**2 / (2 * Cja)
    Ejb = 7 * 1e9 * h
    ECb = 1.0 * 1e9 * h
    Cb = e**2 / (2 * ECb)
    Ib = Ejb / phi0
    Lb = phi0 / Ib
    Ia = Eja / phi0
    Lja = phi0 / Ia
    
    Cg = 14.8e-18
    Eg = e**2 / (2 * Cg)

    # Fit parameters
    gcc = 68.5e6 * h
    nu_R = 6.89e9
    const1 = (np.sqrt((np.pi * 50) / (6.5e3))) * (h * nu_R)
    const = gcc / const1
    Cc = (const * Cb) / (1 - const)
    ECc = e**2 / (2 * Cc)

    return Lja, Cg, Cc, Cja, Lb,Cb,N


## Useful functions

In [3]:
def capacitance_matrix(Cg, Cc, Cja, Cb, N):
    """Calculate and return the capacitance matrix C, setting the last element to zero."""
    C = np.zeros((N, N))
    for i in range(N):
        if i == 0:
            C[i, i] = Cc + Cja + Cg  # Adjust for first node boundary condition
        elif i < N - 1:
            C[i, i] = 2 * Cja + Cg  # Interior nodes
            C[i, i+1] = -Cja
            C[i+1, i] = -Cja
        else:
            # Set the last diagonal element explicitly 
            C[i, i] = Cc + Cja + Cg + Cb   # Last element 

    # Adjust anti-diagonal elements if necessary, while ensuring the last element remains zero
    for i in range(N):
        if i != N - 1:  # Ensure not to alter the last element set previously
            if N % 2 == 1 and (i == N // 2 or i == N - 1 - (N // 2)):
                C[i, N-1-i] = -Cja
            elif N % 2 == 0 and (i == N // 2 - 1 or i == N // 2):
                C[i, N-1-i] = -Cja
            else:
                C[i, N-1-i] = -Cg
        elif i ==N -1:
            C[i,0] = -Cg
        

    return C


def inductance_matrix_inv(Lja, Lb, N):
    """Compute the inductance matrix numerically, setting the last element to zero."""
    L = np.zeros((N, N))
    for i in range(N):
        if i == 0:
            L[i, i] = 1 / Lja   # Adjust for boundary conditions at the first node
        elif i < N - 1:
            L[i, i] = 2 / Lja  # Interior nodes
            L[i, i+1] = -1 / Lja
            L[i+1, i] = -1 / Lja
        else:
            # Ensure the last element remains zero (i == N-1)
            L[i, i] =  1 / Lja + 1 / Lb  # Explicitly set last diagonal element 
            # No need to set off-diagonals as they are already handled by the loop for N-1

    return L

def compute_frequency_eigenvalues(C, L):
    """Calculate frequency eigenvalues from the given capacitance and inductance matrices."""
    C_inv = np.linalg.inv(C)
    C_inv_sqrt = lin.sqrtm(C_inv)
    M = np.dot(np.dot(C_inv_sqrt, L), C_inv_sqrt)
    eigenvalues, eigenvectors = np.linalg.eig(M)
    frequencies = np.sqrt((eigenvalues)) / (2 * np.pi)  # Convert from rad/s to Hz
    return frequencies, eigenvectors

## Plotting eigen values and eigen vectors

In [4]:
# Initialize parameters
Lja, Cg, Cc, Cja, Lb, Cb,N = initialize_parameters()

# Generate capacitance and inductance matrices
C_matrix = capacitance_matrix(Cg, Cc, Cja, Cb, N)  # Assuming you want to use Cg for Cb in your example
L_inv_matrix = inductance_matrix_inv(Lja, Lb, N)

# Compute eigenvalues and eigenvectors
frequencies, eigenvectors = compute_frequency_eigenvalues(C_matrix, L_inv_matrix)

# Plotting the first N/2 + 1 eigenvalues and the first three eigenvectors
plt.figure(figsize=(10, 4))

# Plot eigenvalues
plt.subplot(1, 2, 1)
# plt.semilogx(range(2, N+1), frequencies[1:]/1e9, 'o')  # Convert Hz to GHz
plt.semilogx(range(1, N+1), frequencies/1e9, 'o')  # Convert Hz to GHz

plt.title('Computed Frequencies (GHz)')
plt.xlabel('Mode Number')
plt.ylabel('Frequency (GHz)')
plt.ylim(0,33)
plt.grid(True, which="both", ls="--")

# Plot the first three eigenvectors
plt.subplot(1, 2, 2)
for i in range(4):
    plt.plot(eigenvectors[:int(N/2 + 1), i], label=f'Eigenvector {i + 1}')
plt.title('First four modes')
plt.xlabel('site_index')
plt.ylabel('Amplitude')
plt.legend()

C:\Users\shele\anaconda3\Lib\site-packages\matplotlib\cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
C:\Users\shele\anaconda3\Lib\site-packages\matplotlib\cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [5]:
frequencies

array([2.97091187e+09+0.j, 5.77870376e+09+0.j, 7.25429651e+09+0.j, ...,
       2.94997216e+10+0.j, 2.94997216e+10+0.j, 2.94997216e+10+0.j])

### Plotting eigen values and eigen vectors with phi

In [27]:
# def compute_frequency_eigenvalues_phi(C, L):
#     """Calculate frequency eigenvalues, eigenvectors, and phi from the given capacitance and inductance matrices."""
#     C_inv = np.linalg.inv(C)
#     C_inv_sqrt = lin.sqrtm(C_inv)
#     M = np.dot(np.dot(C_inv_sqrt, L), C_inv_sqrt)
#     eigenvalues, eigenvectors = np.linalg.eig(M)
#     frequencies = np.sqrt(np.abs(eigenvalues)) / (2 * np.pi)  # Convert from rad/s to Hz
#     frequencies = np.real(frequencies)  # Ensure frequencies are real

#     # Compute phi from eigenvectors
#     phi = np.dot(C_inv_sqrt, eigenvectors)
#     phi = 2 * cst.e * phi * np.sqrt(1 / (cst.h * frequencies[np.newaxis, :]))

#     return frequencies, eigenvectors, phi  # Ensure all three values are returned

# # Ensure N is defined and matrices C and L are initialized properly before this step
# N = 1498  # Define the size of matrices
# Cg = 21.6e-18
# Cc = 4.32e-15
# Cja = 11.25e-15
# Cb = 2.66e-15
# Lja = 3.49e-9
# Lb = 33.12e-9

# C_matrix = capacitance_matrix(Cg, Cc, Cja, Cb, N)
# L_inv_matrix = inductance_matrix_inv(Lja, Lb, N)

# # Compute the eigenvalues, eigenvectors, and phi
# frequencies, eigenvectors, phi = compute_frequency_eigenvalues(C_matrix, L_inv_matrix)

# # Plot the phi values for the first three eigenvectors up to N/2
# plt.figure(figsize=(12, 6))
# for i in range(3):
#     plt.plot(phi[:int(N/2+1), i+1], label=f'Phi for Eigenvector {i+1}')
# plt.title('Phi for First Three Eigenvectors (First Half)')
# plt.xlabel('Component Index')
# plt.ylabel('Phi Value')
# plt.legend()
# plt.show()

ValueError: not enough values to unpack (expected 3, got 2)

# RAM sample

In [6]:
from __future__ import division
import numpy as np
from numpy.linalg import inv
import scipy.linalg as lin
import scipy.constants as cst
import matplotlib.pyplot as plt
%matplotlib qt

In [7]:
# Physical constants and parameters
h = cst.h
e = cst.e

In [8]:
def capacitance_matrix(Cg, Cc, Cja, Cb, N):
    """Calculate and return the capacitance matrix C, setting the last element to zero."""
    C = np.zeros((N, N))
    for i in range(N):
        if i == 0:
            C[i, i] = Cc + Cja + Cg  # Adjust for first node boundary condition
        elif i < N - 1:
            C[i, i] = 2 * Cja + Cg  # Interior nodes
            C[i, i+1] = -Cja
            C[i+1, i] = -Cja
        else:
            # Set the last diagonal element explicitly 
            C[i, i] = Cc + Cja + Cg + Cb   # Last element 

    # Adjust anti-diagonal elements if necessary, while ensuring the last element remains zero
    for i in range(N):
        if i != N - 1:  # Ensure not to alter the last element set previously
            if N % 2 == 1 and (i == N // 2 or i == N - 1 - (N // 2)):
                C[i, N-1-i] = -Cja
            elif N % 2 == 0 and (i == N // 2 - 1 or i == N // 2):
                C[i, N-1-i] = -Cja
            else:
                C[i, N-1-i] = -Cg
        elif i ==N -1:
            C[i,0] = -Cg
        

    return C


def inductance_matrix_inv(Lja, Lb, N):
    """Compute the inductance matrix numerically, setting the last element to zero."""
    L = np.zeros((N, N))
    for i in range(N):
        if i == 0:
            L[i, i] = 1 / Lja   # Adjust for boundary conditions at the first node
        elif i < N - 1:
            L[i, i] = 2 / Lja  # Interior nodes
            L[i, i+1] = -1 / Lja
            L[i+1, i] = -1 / Lja
        else:
            # Ensure the last element remains zero (i == N-1)
            L[i, i] =  1 / Lja + 1 / Lb  # Explicitly set last diagonal element 
            # No need to set off-diagonals as they are already handled by the loop for N-1

    return L


def compute_frequency_eigenvalues(C, L):
    """Calculate frequency eigenvalues from the given capacitance and inductance matrices."""
    C_inv = np.linalg.inv(C)
    C_inv_sqrt = lin.sqrtm(C_inv)
    M = np.dot(np.dot(C_inv_sqrt, L), C_inv_sqrt)
    eigenvalues, eigenvectors = np.linalg.eig(M)
    frequencies = np.sqrt((eigenvalues)) / (2 * np.pi)  # Convert from rad/s to Hz
    return frequencies, eigenvectors

In [9]:
# Constants and matrix generation
N = 418 #418
Cja = 11.25e-15
Lja = 3.49e-9
Cg = 21.6e-18 #23.35
Cc = 4.32e-15 # 4.32e-15 
Cb = 2.66e-15 
Lb = 33.12e-9


C_matrix = capacitance_matrix(Cg, Cc, Cja, Cb, N)
L_inv_matrix = inductance_matrix_inv(Lja, Lb, N)
frequencies, eigenvectors = compute_frequency_eigenvalues(C_matrix, L_inv_matrix)

# Plotting the frequencies in one plot
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.semilogx(range(1, N+1), frequencies/1e9, 'o')  # Convert Hz to GHz

plt.title('Computed Frequencies (GHz)')
plt.xlabel('Mode Number')
plt.ylabel('Frequency (GHz)')
plt.ylim(0,26)
plt.grid(True, which="both", ls="--")

# Plotting all three eigenvectors together in another plot
plt.subplot(1, 2, 2)

for i in range(4):
    plt.plot(eigenvectors[:int(N/2 + 1),i], label=f'Eigenvector {i+1}')
plt.title('First four modes')
plt.xlabel('site_index')
plt.ylabel('Amplitude')
plt.legend()
plt.tight_layout()
plt.show()

## Fitting function

In [5]:
from scipy.optimize import minimize
error_history = []  # This list will store the error values

def error_function(params, expt_data, Cja, Lja, Cb, Lb, N):
    Cg, Cc = params
    C_matrix = capacitance_matrix(Cg, Cc, Cja, Cb, N)
    L_inv_matrix = inductance_matrix_inv(Lja, Lb, N)
    frequencies, _ = compute_frequency_eigenvalues(C_matrix, L_inv_matrix)
    computed_freqs = frequencies[:4] / (2 * np.pi)  # Convert first four frequencies from rad/s to Hz
    error = np.sum((computed_freqs - expt_data)**2)
    error_history.append(error)  # Append current error to the history
    return error


In [6]:
from scipy.optimize import minimize

# Initial guesses and setup as before
initial_params = [21.6e-18, 4.32e-15]  # initial guesses for Cg and Cc
bounds = [(1e-19, 1e-16), (1e-16, 1e-13)]  # bounds for the parameters

# Experimental data
expt_data = np.array([5.62e9, 10.04e9, 12.72e9, 15.6e9])  # Given experimental frequencies

# Perform optimization
result = minimize(error_function, initial_params, args=(expt_data, Cja, Lja, Cb, Lb, N), bounds=bounds)
optimized_params = result.x
print("Optimized parameters: Cg = {:.2e}, Cc = {:.2e}".format(optimized_params[0], optimized_params[1]))

Optimized parameters: Cg = 2.16e-17, Cc = 4.32e-15


C:\Users\shele\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:598: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
C:\Users\shele\anaconda3\Lib\site-packages\scipy\optimize\_lbfgsb_py.py:356: ComplexWarning: Casting complex values to real discards the imaginary part
  _lbfgsb.setulb(m, x, low_bnd, upper_bnd, nbd, f, g, factr,


In [7]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(error_history, marker='o', linestyle='-')
plt.title('Optimization Error History')
plt.xlabel('Iteration Number')
plt.ylabel('Error')
plt.grid(True)
plt.show()


C:\Users\shele\anaconda3\Lib\site-packages\matplotlib\cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
C:\Users\shele\anaconda3\Lib\site-packages\matplotlib\cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [8]:
result.success,result.message, result.nit

(True, 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 0)

In [9]:
# Define your initial guess and bounds
initial_guess = [21.6e-18, 4.32e-15]  # initial guesses for Cg and Cc
bounds = [(1e-19, 1e-16), (1e-16, 1e-13)]  # Example bounds

# Assuming these are the values from initialize_parameters or defined elsewhere
Cja = 11.25e-15
Lja = 3.49e-9
Cb = 2.66e-15
Lb = 33.12e-9
N = 418
expt_data = np.array([5.62e9, 10.04e9, 12.72e9, 15.6e9])

# Use scipy.optimize.minimize with extra arguments passed
result = minimize(
    error_function, 
    initial_guess, 
    args=(expt_data, Cja, Lja, Cb, Lb, N),  # Pass additional arguments needed by error_function
    method='L-BFGS-B', 
    bounds=bounds,
    options={'ftol': 1e-9, 'gtol': 1e-6}
)

if result.success:
    print("Optimization was successful. Final parameters found.")
    print("Optimized Cg: {:.2e}, Optimized Cc: {:.2e}".format(result.x[0], result.x[1]))
else:
    print("Optimization failed:", result.message)

print("Final error:", result.fun)

Optimization was successful. Final parameters found.
Optimized Cg: 2.16e-17, Optimized Cc: 4.32e-15
Final error: (4.006167936406437e+20+0j)


# Printing capcaitence and inductance matrices

In [12]:
import sympy as sp
import numpy as np

def symbolic_capacitance_matrix(Cg, Cc, Cja, Cb, N):
    """Calculate and return the symbolic capacitance matrix C, setting the last element to zero."""
    C = sp.Matrix.zeros(N, N)
    for i in range(N):
        if i == 0:
            C[i, i] = Cc + Cja + Cg  # Adjust for first node boundary condition
        elif i < N - 1:
            C[i, i] = 2 * Cja + Cg  # Interior nodes
            C[i, i+1] = -Cja
            C[i+1, i] = -Cja
        else:
            # Set the last diagonal element explicitly 
            C[i, i] = Cc + Cja + Cg + Cb   # Last element 

    # Adjust anti-diagonal elements if necessary, while ensuring the last element remains zero
    for i in range(N):
        if i != N - 1:  # Ensure not to alter the last element set previously
            if N % 2 == 1 and (i == N // 2 or i == N - 1 - (N // 2)):
                C[i, N-1-i] = -Cja
            elif N % 2 == 0 and (i == N // 2 - 1 or i == N // 2):
                C[i, N-1-i] = -Cja
            else:
                C[i, N-1-i] = -Cg
        elif i == N - 1:
            C[i, 0] = -Cg
        
    return C

def symbolic_inductance_matrix_inv(Lja, Lb, N):
    """Compute the symbolic inductance matrix numerically, setting the last element to zero."""
    L = sp.Matrix.zeros(N, N)
    for i in range(N):
        if i == 0:
            L[i, i] = 1 / Lja   # Adjust for boundary conditions at the first node
        elif i < N - 1:
            L[i, i] = 2 / Lja  # Interior nodes
            L[i, i+1] = -1 / Lja
            L[i+1, i] = -1 / Lja
        else:
            # Ensure the last element remains zero (i == N-1)
            L[i, i] =  1 / Lja + 1 / Lb  # Explicitly set last diagonal element 
            # No need to set off-diagonals as they are already handled by the loop for N-1

    return L

# Define symbolic variables
Cg, Cc, Cja, Cb, Lja, Lb = sp.symbols('Cg Cc Cja Cb Lja Lb')
N = 5  # Example size of the matrix

# Get symbolic matrices
C_matrix = symbolic_capacitance_matrix(Cg, Cc, Cja, Cb, N)
L_matrix_inv = symbolic_inductance_matrix_inv(Lja, Lb, N)

# Print symbolic matrices
sp.pretty_print(C_matrix)
sp.pretty_print(L_matrix_inv)
# 

⎡Cc + Cg + Cja      0        0        0              -Cg        ⎤
⎢                                                               ⎥
⎢      0        Cg + 2⋅Cja  -Cja     -Cg              0         ⎥
⎢                                                               ⎥
⎢      0           -Cja     -Cja     -Cja             0         ⎥
⎢                                                               ⎥
⎢      0           -Cg      -Cja  Cg + 2⋅Cja         -Cja       ⎥
⎢                                                               ⎥
⎣     -Cg           0        0       -Cja     Cb + Cc + Cg + Cja⎦


In [11]:
sp.pretty_print(L_matrix_inv)


⎡ 1                          ⎤
⎢───   0    0    0      0    ⎥
⎢Lja                         ⎥
⎢                            ⎥
⎢      2   -1                ⎥
⎢ 0   ───  ───   0      0    ⎥
⎢     Lja  Lja               ⎥
⎢                            ⎥
⎢     -1    2   -1           ⎥
⎢ 0   ───  ───  ───     0    ⎥
⎢     Lja  Lja  Lja          ⎥
⎢                            ⎥
⎢          -1    2     -1    ⎥
⎢ 0    0   ───  ───    ───   ⎥
⎢          Lja  Lja    Lja   ⎥
⎢                            ⎥
⎢               -1    1    1 ⎥
⎢ 0    0    0   ───  ─── + ──⎥
⎣               Lja  Lja   Lb⎦


In [13]:
sp.pretty_print(C_matrix)

⎡Cc + Cg + Cja      0        0        0              -Cg        ⎤
⎢                                                               ⎥
⎢      0        Cg + 2⋅Cja  -Cja     -Cg              0         ⎥
⎢                                                               ⎥
⎢      0           -Cja     -Cja     -Cja             0         ⎥
⎢                                                               ⎥
⎢      0           -Cg      -Cja  Cg + 2⋅Cja         -Cja       ⎥
⎢                                                               ⎥
⎣     -Cg           0        0       -Cja     Cb + Cc + Cg + Cja⎦
